# Hướng dẫn truy vấn dữ liệu thị giác dùng fiftyone

Đây là hướng dẫn dùng cho các đội tham dự AI Challenge 2023. Hướng dẫn này nhằm mục đích giới thiệu cho các đội một phương pháp cơ bản để truy vấn dữ liệu dựa trên thông tin BTC cung cấp và giới thiệu công cụ fiftyone để hỗ trợ đội thi đánh giá kết quả.

## Cài đặt ban đầu

Bạn cần cài đặt môi trường để chạy được notebook này trên máy tính cá nhân của bạn. Hướng dẫn này không bao gồm phần cài đặt môi trường. Khuyến nghị: các bạn có thể cài đặt [Anaconda](https://docs.anaconda.com/free/anaconda/install/windows/).

## Cài đặt các thư viện FiftyOne và PyTorch
Hướng dẫn này dùng fiftyone là công cụ để trực quan dữ liệu và pytorch là backend chính cho các thuật toán máy học.

### Lưu ý: Đối với các bạn dùng Windows nên dùng bản fiftyone **v0.21.4**, không nên dùng bản mới nhất!

In [1]:
! pip install fiftyone==0.21.4
! pip install torch torchvision torchaudio


Load dữ liệu keyframe từ thư mục chứa keyframe. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một Sample. Tất cả các Sample được lưu trong Dataset.

In [2]:
import fiftyone as fo
import fiftyone.brain as fob
import fiftyone.zoo as foz
import numpy as np
from glob import glob
import json
import os



Load dữ liệu keyframe từ thư mục chứa keyframe. Trong hướng dẫn này tất cả các file Keyframes_L*.zip được giải nén vào thư mục `D:\AIC\Keyframes`. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một `Sample`. Tất cả các `Sample` được lưu trong `Dataset`. 

In [3]:
# dataset = fo.Dataset.from_images_dir('D:\\CS\\2023 HCM AI CHALLENGE\\keyframes', name=None, tags=None, recursive=True)
dataset = fo.load_dataset("aic2023-full")

Sau khi dữ liệu đã load lên xong. Bạn có thể truy cập vào đường vào ứng dụng web của fiftyone từ [http://localhost:5151](http://localhost:5151)

In [3]:
session = fo.launch_app(dataset, auto=False)


Could not connect session, trying again in 10 seconds

Session launched. Run `session.show()` to open the App in a cell output.


Hoặc bạn có thể chạy cell bên dưới để mở tab mới cho ứng dụng web fiftyone

In [ ]:
# session.open_tab()

### Trích xuất thêm thông tin tên của video và frameid
Thông tin `video` và `frameid` sẽ được lấy từ tên của tập tin keyframe.

In [4]:
for sample in dataset:
    _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('\\', 2)
    sample.save()

Bạn có thể xem `Sample` đầu tiên của `Dataset` bằng lệnh sau:

In [5]:
print(dataset.first())

<Sample: {
    'id': '650556517d3f5eb59773f2cd',
    'media_type': 'image',
    'filepath': 'E:\\AIChallenge\\Keyframes\\L01_V001\\0001.jpg',
    'tags': [],
    'metadata': None,
    'video': 'L01_V001',
    'frameid': '0001',
}>


### Thêm thông tin kết quả của object detection.

Bước này có thể tốn của bạn nhiều thời gian để đọc hết tất cả các dữ liệu về object detection. Bạn có thể bỏ qua cell này và chạy cell này sau nếu muốn thử thêm các thông tin về vector CLIP embedding trước.

In [ ]:
# for sample in dataset:
#     object_path = f"...\\objects\\{sample['video']}\\{sample['frameid']}.json"
#     with open(object_path) as jsonfile:
#         det_data = json.load(jsonfile)
#     detections = []
#     for cls, box, score in zip(det_data['detection_class_entities'], det_data['detection_boxes'], det_data['detection_scores']):
#         # Convert to [top-left-x, top-left-y, width, height]
#         boxf = [float(box[1]), float(box[0]), float(box[3]) - float(box[1]), float(box[2]) - float(box[0])]
#         scoref = float(score)
        
#         # Only add objects with confidence > 0.4
#         if scoref > 0.4:
#             detections.append(
#                 fo.Detection(
#                     label=cls,
#                     bounding_box= boxf,
#                     confidence=float(score)
#                 )
#             )
#     sample["object_faster_rcnn"] = fo.Detections(detections=detections)
#     sample.save()
                           

### Thêm thông tin CLIP embedding.

In [6]:
all_keyframe = glob('E:\\AIChallenge\\Keyframes\\*\\*.jpg')
video_keyframe_dict = {}
all_video = glob('E:\\AIChallenge\\Keyframes\\*')
all_video = [v.rsplit('\\', 1)[-1] for v in all_video]
print(all_video)

['L01_V001', 'L01_V002', 'L01_V003', 'L01_V004', 'L01_V005', 'L01_V006', 'L01_V007', 'L01_V008', 'L01_V009', 'L01_V010', 'L01_V011', 'L01_V012', 'L01_V013', 'L01_V014', 'L01_V015', 'L01_V016', 'L01_V017', 'L01_V018', 'L01_V019', 'L01_V020', 'L01_V021', 'L01_V022', 'L01_V023', 'L01_V024', 'L01_V025', 'L01_V026', 'L01_V027', 'L01_V028', 'L01_V029', 'L01_V030', 'L01_V031', 'L02_V002', 'L02_V003', 'L02_V004', 'L02_V005', 'L02_V006', 'L02_V007', 'L02_V008', 'L02_V009', 'L02_V010', 'L02_V011', 'L02_V012', 'L02_V013', 'L02_V014', 'L02_V015', 'L02_V016', 'L02_V017', 'L02_V018', 'L02_V019', 'L02_V020', 'L02_V021', 'L02_V022', 'L02_V023', 'L02_V024', 'L02_V025', 'L02_V026', 'L02_V027', 'L02_V028', 'L02_V029', 'L02_V030', 'L02_V001', 'L03_V002', 'L03_V003', 'L03_V004', 'L03_V005', 'L03_V006', 'L03_V007', 'L03_V008', 'L03_V009', 'L03_V010', 'L03_V011', 'L03_V012', 'L03_V013', 'L03_V014', 'L03_V015', 'L03_V016', 'L03_V017', 'L03_V018', 'L03_V019', 'L03_V020', 'L03_V021', 'L03_V022', 'L03_V023', 'L0

Đọc thông tin clip embedding được cung cấp.

Lưu ý: Các bạn cần tải đúng bản CLIP embedding từ model **CLIP ViT-B/32**

Tạo dictionary `video_keyframe_dict` với `video_keyframe_dict[video]` thông tin danh sách `keyframe` của `video` 

In [7]:
for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('\\',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

Do thông tin vector CLIP embedding được cung cấp được lưu theo từng video nhầm mục đích tối ưu thời gian đọc dữ liệu. Cần sort lại danh sách `keyframe` của từng `video` để đảm bảo thứ tự đọc đúng với vector embedding được cung cấp.

In [8]:
for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

Tạo dictionary `embedding_dict` với `embedding_dict[video][keyframe]` lưu thông tin vector CLIP embedding của `keyframe` trong `video` tương ứng

In [11]:
embedding_dict = {}
for v in all_video:
    clip_path = f'E:\\AIChallenge\\clip-features-vit-b32\\{v}.npy'
    a = np.load(clip_path)
    embedding_dict[v] = {}
    for i,k in enumerate(video_keyframe_dict[v]):
        embedding_dict[v][k] = a[i]
    print(clip_path)

E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V001.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V002.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V003.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V004.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V005.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V006.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V007.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V008.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V009.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V010.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V011.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V012.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V013.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V014.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V015.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V016.npy
E:\2023 HCM AI CHALLENGE\clip-features-v

Tạo danh sách `clip_embedding` ứng với danh sách `sample` trong `dataset`.

In [12]:
clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['frameid']]
    clip_embeddings.append(clip_embedding)

In [ ]:
# Này là tạo embeddings với backend mặc định là pytorch -> search lâu hơn qdrant
# image_index = fob.compute_similarity(
#     dataset,
#     model="clip-vit-base32-torch",      # store model's name for future use
#     embeddings=clip_embeddings,          # precomputed image embeddings    
#     brain_key="img_sim_32",
# )

## Từ đây các bạn có thể thử các tính năng search, filter trên ứng dụng fiftyone.

#### **Qdrant integration**

In [15]:
# Nếu trước đó có tạo 1 bộ embeddings với brain_key là "img_sim_32_qdrant" thì phải xóa để tạo lại
fob.similarity.Similarity.delete_run(dataset, "img_sim_32_qdrant")

In [16]:
qdrant_index = fob.compute_similarity(
    dataset, 
    model = "clip-vit-base32-torch",     
    embeddings=clip_embeddings,          # precomputed image embeddings  
    brain_key = "img_sim_32_qdrant", 
    backend="qdrant",
    metric="cosine",
    collection_name = "aic2023-L01-L20"
)
dataset.save()

#### **Embed text prompts**

In [ ]:
import numpy as np
from pkg_resources import packaging
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

if packaging.version.parse(
  torch.__version__
) < packaging.version.parse("1.8.0"):
  dtype = torch.long
else:
  dtype = torch.int

# Hàm này dùng để tính embedding của text để từ đó so sánh với embedding của image, thay vì chỉ quăng cục text vô hàm compute_similarity rồi cho nó search mà ko biết embedding của text là gì
# Cơ bản là Q muốn thử xem 2 cách có gì khác nhau ko và đáp án là 0

def get_text_embedding(prompt, clip_model):
    tokenizer = clip_model._tokenizer

    # standard start-of-text token
    sot_token = tokenizer.encoder["<|startoftext|>"]

    # standard end-of-text token
    eot_token = tokenizer.encoder["<|endoftext|>"]

    prompt_tokens = tokenizer.encode(prompt)
    all_tokens = [[sot_token] + prompt_tokens + [eot_token]]

    text_features = torch.zeros(
        len(all_tokens),
        clip_model.config.context_length,
        dtype=dtype,
        device=device,
    )

    # insert tokens into feature vector
    text_features[0, : len(all_tokens[0])] = torch.tensor(all_tokens)

    # encode text
    embedding = clip_model._model.encode_text(text_features).to(device)

    # convert to list for Pinecone
    return embedding.tolist()

In [17]:
# 2 stages

from fiftyone import ViewField as F

# model = foz.load_zoo_model("clip-vit-base32-torch")
# prompt = "Sunset over city skyline, golden hues."
# query_vector = get_text_embedding(prompt, model)
# stage = dataset.sort_by_similarity(query_vector[0], k=16, brain_key="img_sim_32_qdrant")

query1 = "a laptop with some mobile phones on a table"
query2 = "laptop AND mobile phones"
view1 = (dataset
        .sort_by_similarity(query1, k=25, brain_key="img_sim_32_qdrant")     
        .sort_by_similarity(query2, k=25, brain_key="img_sim_32_qdrant")               
        # .filter_labels("object_faster_rcnn", F("label").is_in(["Mobile phone"]))
)

for sample in view1:
    print(f"{sample.video}_{sample.frameid}")

session.view = view1.view()

L01_V009_0139
L01_V005_0130
L01_V009_0104
L01_V009_0103
L02_V005_0219
L02_V005_0218
L01_V022_0099
L02_V025_0106
L02_V005_0171
L01_V016_0120
L01_V025_0154
L01_V012_0109
L02_V025_0107
L02_V005_0187
L01_V012_0092
L02_V015_0226
L02_V001_0263
L02_V022_0192
L02_V018_0064
L02_V017_0062
L01_V022_0098
L02_V018_0325
L02_V011_0034
L02_V010_0229
L02_V024_0315


In [36]:
query2 = "only one man in green clothes"
view2 = (dataset
        .sort_by_similarity(query2, k=25, brain_key="img_sim_32_qdrant")       
)

for sample in view2:
    print(f"{sample.video}_{sample.frameid}")

session.view = view2.view()

L01_V027_0089
L02_V017_0205
L02_V001_0262
L01_V013_0110
L02_V011_0010
L02_V020_0223
L02_V020_0010
L02_V009_0235
L01_V019_0085
L02_V020_0224
L02_V008_0202
L01_V019_0086
L02_V005_0242
L02_V020_0232
L01_V019_0093
L01_V030_0159
L02_V019_0186
L01_V013_0036
L01_V019_0094
L02_V023_0249
L02_V012_0018
L02_V001_0041
L02_V001_0257
L02_V013_0256
L02_V001_0210


In [ ]:
# Này là gì cũng chưa có test thử á
fob.compute_similarity(
    dataset, 
    model="clip-vit-base32-torch",
    patches_field="object_faster_rcnn",
    brain_key = "qdrant_clip_patches", 
    backend="qdrant",
    metric="cosine",
    collection_name="fiftyone-patches"
)

In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

import fiftyone as fo
import fiftyone.brain as fob
import fiftyone.zoo as foz
from fiftyone import ViewField as F
import numpy as np
from glob import glob
import json
import os

import heapq
from mapping_keyframe import get_frame_info, parse_direc
from extract_frame import extract_frames


In [2]:
dataset = fo.load_dataset('aic2023-kf-full-1')

# for sample in dataset:
#     _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('\\', 2)
#     sample.save()

all_keyframe = glob('E:\\AIChallenge\\Keyframes\\*\\*.jpg')
video_keyframe_dict = {}
all_video = glob('E:\\AIChallenge\\Keyframes\\*')
all_video = [v.rsplit('\\', 1)[-1] for v in all_video]
# print(all_keyframe)
print(all_video)

for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('\\',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)




['L01_V001', 'L01_V002', 'L01_V003', 'L01_V004', 'L01_V005', 'L01_V006', 'L01_V007', 'L01_V008', 'L01_V009', 'L01_V010', 'L01_V011', 'L01_V012', 'L01_V013', 'L01_V014', 'L01_V015', 'L01_V016', 'L01_V017', 'L01_V018', 'L01_V019', 'L01_V020', 'L01_V021', 'L01_V022', 'L01_V023', 'L01_V024', 'L01_V025', 'L01_V026', 'L01_V027', 'L01_V028', 'L01_V029', 'L01_V030', 'L01_V031', 'L02_V002', 'L02_V003', 'L02_V004', 'L02_V005', 'L02_V006', 'L02_V007', 'L02_V008', 'L02_V009', 'L02_V010', 'L02_V011', 'L02_V012', 'L02_V013', 'L02_V014', 'L02_V015', 'L02_V016', 'L02_V017', 'L02_V018', 'L02_V019', 'L02_V020', 'L02_V021', 'L02_V022', 'L02_V023', 'L02_V024', 'L02_V025', 'L02_V026', 'L02_V027', 'L02_V028', 'L02_V029', 'L02_V030', 'L02_V001', 'L03_V002', 'L03_V003', 'L03_V004', 'L03_V005', 'L03_V006', 'L03_V007', 'L03_V008', 'L03_V009', 'L03_V010', 'L03_V011', 'L03_V012', 'L03_V013', 'L03_V014', 'L03_V015', 'L03_V016', 'L03_V017', 'L03_V018', 'L03_V019', 'L03_V020', 'L03_V021', 'L03_V022', 'L03_V023', 'L0

In [3]:
embedding_dict = {}
for v in all_video:
    # print(v)
    clip_path = f'E:\\AIChallenge\\clip-features-vit-b32\\{v}.npy'
    a = np.load(clip_path)
    embedding_dict[v] = {}
    for i,k in enumerate(video_keyframe_dict[v]):
        embedding_dict[v][k] = a[i]
        # print(i, k, a[i])

clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['frameid']]
    clip_embeddings.append(clip_embedding)

# fob.compute_similarity(
#     dataset,
#     model="clip-vit-base32-torch",      # store model's name for future use
#     embeddings=clip_embeddings,          # precomputed image embeddings
#     brain_key="img_sim_32_qdrant",
# )


fob.similarity.Similarity.delete_run(dataset, "img_sim_32_qdrant")
# if fob.similarity.Similarity.has_cached_run_results(dataset, "img_sim_32_qdrant"):
#     fob.similarity.Similarity.delete_run(dataset, "img_sim_32_qdrant")

qdrant_index = fob.compute_similarity(
    dataset, 
    model = "clip-vit-base32-torch",     
    embeddings=clip_embeddings,          # precomputed image embeddings  
    brain_key = "img_sim_32_qdrant", 
    backend="qdrant",
    metric="cosine",
    collection_name = "aic2023-L01-L20"
)
dataset.save()

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Failed to connect to Qdrant backend at URL 'None'. Refer to https://docs.voxel51.com/integrations/qdrant.html for more information